# Deliverable 1. Retrieve Weather Data.
---
1. Create a folder called `Weather_Database` to save all the files related with this deliverable.

2. Save the `Weather_Database_starter_code.ipynb` starter code to the `Weather_Database` folder and rename it as `Weather_Database.ipynb`.

3. Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

4. Use the `citipy` module to get the nearest city for each latitude and longitude combination.

5. Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key; also, it's critical to avoid publishing your API key on your GitHub repository.

6. Retrieve the following information from the API call:

    * Latitude and longitude

    * Maximum temperature

    * Percent humidity

    * Percent cloudiness

    * Wind speed

    * Weather description (for example, clouds, fog, light rain, clear sky)

7. Add the weather data to a new DataFrame.

8. Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.
---

In [13]:
# Import initial libraries
import numpy as np
import pandas as pd
from citipy import citipy

## Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

In [14]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)

# Use the zip function to create an iterator of tuples containing the latitude and longitude combinations
lat_lngs = zip(lats, lngs)
lat_lngs

In [15]:
# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

15.495804539567075 -163.0669034098159
87.19134683607851 123.9253159072253
48.92794737299812 -63.6632537177409
-0.8486200747212678 49.334473728830545
-3.2125495580534107 -17.875872848555787
-7.520133958973176 131.2923610163665
6.816590759883383 161.89961432677597
-36.58038766041612 93.98344278750352
26.438853136602646 -95.45095638196375
-22.95811846993122 -1.9116627452348496
-1.9863148527146848 -45.79180711472176
-51.3511689105049 41.946725294464756
-33.65160602471862 3.2668309215402758
-35.15355416087702 -75.604307111364
-36.09872791970817 -92.52889673223136
8.093107011760551 113.72630033004879
-14.05875005722092 149.61970441007549
8.569312913096951 53.07887002039894
-38.573910769316186 -147.606389789776
12.592603647878306 -138.66096257178776
-68.43619582325843 -116.4466428433987
-7.107922651434805 -86.73139118323765
49.913829269448286 -31.262733847002664
-78.88737603879092 129.14581168027667
-40.09085391057705 -132.08264532011677
11.455492307519478 -59.32356044928737
-82.0380001767112

## Use the `citipy` module to get the nearest city for each latitude and longitude combination.

In [16]:
# Create an empty list for holding the cities
cities = []

# Use a for loop to identify nearest city for each latitude and longitude combination using the citipy module
for coordinate in coordinates:
    # Use the citipy module to identify the nearest city to the current coordinate
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

807

## Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key.

In [18]:
# Import the requests library
import requests

# Import the time library
import time

# Import the datetime module from the datetime library
from datetime import datetime

# Import the OpenWeatherMap's API key
from config import weather_api_key

In [19]:
# Assemble the OpenWeatherMap's API call



url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [34]:
city_url = url + "&q=" + "Boston"
city_weather = requests.get(city_url)
city_weather.json()["weather"][0]['description']

'mist'

## Retrieve the following information from the API call.

- The latitude and longitude
- The Max temperature
- The % humidity
- The % cloudiness
- The Wind speed
- The Weather description, i.e., cloudy, fog, light rain, clear sky, etc. 

In [37]:
# Create an empty list to hold weather data for each city
city_data = []

# Print a message to indicate that the data retrieval starts
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters and set them to 1
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data for each city
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create an endpoint URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        city_weather = requests.get(city_url).json()
        # Parse out the latitude, longitude, max temp, humidity, cloudiness, wind, country, and weather description
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_desc = city_weather["weather"][0]['description']
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
            
        # Append the city information into the city_data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Decription": city_desc ,
                          "Date": city_date})
    
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

# Indicate that the data retrieval is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | kapaa
Processing Record 2 of Set 1 | tiksi
Processing Record 3 of Set 1 | riviere-au-renard
Processing Record 4 of Set 1 | mogadishu
Processing Record 5 of Set 1 | georgetown
Processing Record 6 of Set 1 | tual
Processing Record 7 of Set 1 | butaritari
Processing Record 8 of Set 1 | busselton
Processing Record 9 of Set 1 | brownsville
Processing Record 10 of Set 1 | jamestown
Processing Record 11 of Set 1 | maracacume
Processing Record 12 of Set 1 | east london
Processing Record 13 of Set 1 | luderitz
Processing Record 14 of Set 1 | talcahuano
Processing Record 15 of Set 1 | lebu
Processing Record 16 of Set 1 | kota kinabalu
Processing Record 17 of Set 1 | samarai
Processing Record 18 of Set 1 | bandarbeyla
Processing Record 19 of Set 1 | mataura
Processing Record 20 of Set 1 | hilo
Processing Record 21 of Set 1 | punta arenas
Processing Record 22 of Set 1 | paita
Processing Record 23 of Set 1 | 

Processing Record 36 of Set 4 | marataizes
Processing Record 37 of Set 4 | salalah
Processing Record 38 of Set 4 | queanbeyan
Processing Record 39 of Set 4 | westport
Processing Record 40 of Set 4 | ipixuna
Processing Record 41 of Set 4 | kalamb
Processing Record 42 of Set 4 | bara
Processing Record 43 of Set 4 | wanning
Processing Record 44 of Set 4 | fereydun kenar
Processing Record 45 of Set 4 | craig
Processing Record 46 of Set 4 | pochutla
Processing Record 47 of Set 4 | tasiilaq
Processing Record 48 of Set 4 | winona
Processing Record 49 of Set 4 | ponta do sol
Processing Record 50 of Set 4 | klaksvik
Processing Record 1 of Set 5 | bjornevatn
Processing Record 2 of Set 5 | ilulissat
Processing Record 3 of Set 5 | santa clara
Processing Record 4 of Set 5 | vardo
Processing Record 5 of Set 5 | kangaba
Processing Record 6 of Set 5 | la ronge
Processing Record 7 of Set 5 | bargal
City not found. Skipping...
Processing Record 8 of Set 5 | mahebourg
Processing Record 9 of Set 5 | gravd

Processing Record 29 of Set 8 | la romana
Processing Record 30 of Set 8 | marsh harbour
Processing Record 31 of Set 8 | portland
Processing Record 32 of Set 8 | alghero
Processing Record 33 of Set 8 | semey
Processing Record 34 of Set 8 | vostok
Processing Record 35 of Set 8 | saint-georges
Processing Record 36 of Set 8 | nagapattinam
Processing Record 37 of Set 8 | bilibino
Processing Record 38 of Set 8 | belushya guba
City not found. Skipping...
Processing Record 39 of Set 8 | cheuskiny
City not found. Skipping...
Processing Record 40 of Set 8 | deputatskiy
Processing Record 41 of Set 8 | lasa
Processing Record 42 of Set 8 | bukama
Processing Record 43 of Set 8 | campbell river
Processing Record 44 of Set 8 | maku
Processing Record 45 of Set 8 | mardin
Processing Record 46 of Set 8 | grajau
Processing Record 47 of Set 8 | tautira
Processing Record 48 of Set 8 | tanggu
Processing Record 49 of Set 8 | sychevka
Processing Record 50 of Set 8 | chifeng
Processing Record 1 of Set 9 | ancud

Processing Record 14 of Set 12 | emirdag
Processing Record 15 of Set 12 | labrea
Processing Record 16 of Set 12 | belaya gora
Processing Record 17 of Set 12 | banyo
Processing Record 18 of Set 12 | gornopravdinsk
Processing Record 19 of Set 12 | port hedland
Processing Record 20 of Set 12 | kilindoni
Processing Record 21 of Set 12 | mujiayingzi
Processing Record 22 of Set 12 | beyneu
Processing Record 23 of Set 12 | kirando
Processing Record 24 of Set 12 | cururupu
Processing Record 25 of Set 12 | cruzeiro do sul
Processing Record 26 of Set 12 | cerritos
Processing Record 27 of Set 12 | urengoy
Processing Record 28 of Set 12 | charters towers
Processing Record 29 of Set 12 | ushtobe
Processing Record 30 of Set 12 | leshan
Processing Record 31 of Set 12 | dubbo
Processing Record 32 of Set 12 | maniitsoq
Processing Record 33 of Set 12 | constitucion
Processing Record 34 of Set 12 | moa
Processing Record 35 of Set 12 | seybaplaya
Processing Record 36 of Set 12 | mount isa
Processing Recor

Processing Record 1 of Set 16 | repnoye
Processing Record 2 of Set 16 | bulungu
Processing Record 3 of Set 16 | blenheim
Processing Record 4 of Set 16 | zhigansk
Processing Record 5 of Set 16 | chapais
Processing Record 6 of Set 16 | ixtapa
Processing Record 7 of Set 16 | skegness
Processing Record 8 of Set 16 | ternate
Processing Record 9 of Set 16 | nassau
Processing Record 10 of Set 16 | anloga
Processing Record 11 of Set 16 | hays
Processing Record 12 of Set 16 | ngunguru
Processing Record 13 of Set 16 | mocuba
Processing Record 14 of Set 16 | duluth
Processing Record 15 of Set 16 | amurzet
Processing Record 16 of Set 16 | seydisehir
Processing Record 17 of Set 16 | la reforma
Processing Record 18 of Set 16 | alice
Processing Record 19 of Set 16 | kirakira
Processing Record 20 of Set 16 | sorland
Processing Record 21 of Set 16 | durazno
Processing Record 22 of Set 16 | manokwari
Processing Record 23 of Set 16 | novopetrovskoye
Processing Record 24 of Set 16 | jiexiu
Processing Reco

In [38]:
# Print the length of the city_data list to verify how many cities you have
len(city_data)

748

## Add the weather data to a new DataFrame.

In [39]:
# Use the city_data list to create a new pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

In [40]:
# Display sample data
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Decription,Date
0,Kapaa,22.0752,-159.3190,80.58,88,100,20.71,US,light rain,2022-12-12 01:45:04
1,Tiksi,71.6872,128.8694,-18.71,95,100,12.08,RU,overcast clouds,2022-12-12 01:49:21
2,Riviere-Au-Renard,48.9957,-64.3971,12.49,72,92,12.57,CA,overcast clouds,2022-12-12 01:49:21
3,Mogadishu,2.0371,45.3438,77.99,80,50,17.31,SO,scattered clouds,2022-12-12 01:49:22
4,Georgetown,5.4112,100.3354,80.53,82,20,4.61,MY,few clouds,2022-12-12 01:46:11
5,Tual,-5.6667,132.7500,77.67,87,100,4.47,ID,overcast clouds,2022-12-12 01:49:22
6,Butaritari,3.0707,172.7902,80.83,73,100,11.03,KI,overcast clouds,2022-12-12 01:49:22
7,Busselton,-33.6500,115.3333,69.67,38,0,11.52,AU,clear sky,2022-12-12 01:49:23
8,Brownsville,25.9018,-97.4975,74.08,89,75,5.75,US,broken clouds,2022-12-12 01:49:23
9,Jamestown,42.0970,-79.2353,34.02,72,100,6.91,US,mist,2022-12-12 01:45:24


In [41]:
# Display the DataFrame's column names using the columns Pandas function
city_data_df.columns

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Decription', 'Date'],
      dtype='object')

In [44]:
# Create a list to reorder the column names as follows:
# "City", "Country", "Lat", "Lng", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Current Description"
column_order = ['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Decription']

# Recreate the DataFrame by using the new column order
city_data_df = city_data_df[column_order]

# Display sample data
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Decription
0,Kapaa,US,22.0752,-159.3190,80.58,88,100,20.71,light rain
1,Tiksi,RU,71.6872,128.8694,-18.71,95,100,12.08,overcast clouds
2,Riviere-Au-Renard,CA,48.9957,-64.3971,12.49,72,92,12.57,overcast clouds
3,Mogadishu,SO,2.0371,45.3438,77.99,80,50,17.31,scattered clouds
4,Georgetown,MY,5.4112,100.3354,80.53,82,20,4.61,few clouds
5,Tual,ID,-5.6667,132.7500,77.67,87,100,4.47,overcast clouds
6,Butaritari,KI,3.0707,172.7902,80.83,73,100,11.03,overcast clouds
7,Busselton,AU,-33.6500,115.3333,69.67,38,0,11.52,clear sky
8,Brownsville,US,25.9018,-97.4975,74.08,89,75,5.75,broken clouds
9,Jamestown,US,42.0970,-79.2353,34.02,72,100,6.91,mist


In [48]:
# Display the data types of each column by using the dtypes Pandas function
city_data_df.dtypes

City           object
Country        object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Decription     object
dtype: object

## Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.

In [50]:
# Set the output file name
output_data_file = '../Weather_Database/WeatherPy_Database.csv'

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")